In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import pickle

import matplotlib.pyplot as plt
from cycler import cycler

### Read the parsed results

In [ ]:
dict_pwr_final_res = None
dict_frag_final_res = None
dict_sched_final_res = None
dict_efficiency = None

with open('dict_pwr_final_res.pkl', 'rb') as f:
    dict_pwr_final_res = pickle.load(f)

with open('dict_frag_final_res.pkl', 'rb') as f:
    dict_frag_final_res = pickle.load(f)

with open('dict_sched_final_res.pkl', 'rb') as f:
    dict_sched_final_res = pickle.load(f)

with open('dict_efficiency.pkl', 'rb') as f:
    dict_efficiency = pickle.load(f)

# Generation of latex tables

In [ ]:
import re

def split_string(s):
    match = re.match(r"([a-zA-Z_]+)(\d+)", s)
    if match:
        return match.groups()
    else:
        return s, ""

In [ ]:
# Generate latex tables for GPU efficiency
df_eff = {}
for level in dict_efficiency.keys() :

    exp, perc = split_string(level)
    df = df_eff.get(exp, pd.DataFrame(index = pd.Index([], name='Scheduling Policy')))

    if perc != "" : perc = f' ({perc}\%)'
    for policy in dict_efficiency[level].keys() :
        df.loc[policy, f'GRAR{perc}'] = dict_efficiency[level][policy]['usage_efficiency'].loc[1]

    df_eff[exp] = df

display(df_eff['openb_pod_list_gpushare'])

In [ ]:
for key, df in df_eff.items():
    print(f'Generating latex table for {key}...')
    
    col_format = 'c' * (df.shape[1] + 1)
    df_temp = df.copy()
    
    df_temp.index = pd.Index([x.replace('_', '\_') for x in df_temp.index.values], name = df_temp.index.name)
    
    # Reset the index
    df_temp.reset_index(inplace = True)

    # Ensure that the first column has the name of the old index
    df_temp.columns = [f'\\textbf{{{x}}}' for x in df_temp.columns.values]

    # Boldify the names of the policies.
    df_temp.iloc[:, 0] = [f'\\textbf{{{x}}}' for x in df_temp.iloc[:, 0]]
    
    print(df_temp.to_latex(column_format = col_format, float_format="%.3f", index = False))